In [4]:
import pandas as pd

In [5]:
df = pd.read_json('{"l1":{"0":"a","1":"a","2":"a","3":"a","4":"b","5":"b","6":"b","7":"b"},"l2":{"0":"a1","1":"a1","2":"a2","3":"a2","4":"b1","5":"b1","6":"b2","7":"b3"},"l3":{"0":"a11","1":"a12","2":"a21","3":"a22","4":"b11","5":"b12","6":"b22","7":"b34"},"val":{"0":1,"1":2,"2":3,"3":4,"4":5,"5":6,"6":7,"7":8}}')
df

,l1,l2,l3,val
0,a,a1,a11,1
1,a,a1,a12,2
2,a,a2,a21,3
3,a,a2,a22,4
4,b,b1,b11,5
5,b,b1,b12,6
6,b,b2,b22,7
7,b,b3,b34,8


In [6]:
d = {"name": "root",
"children": []}

In [7]:
def find_element(children_list,name):
    """
    Find element in children list
    if exists or return none
    """
    for i in children_list:
        if i["name"] == name:
            return i
    #If not found return None
    return None

def add_node(path,value,nest):
    """
    The path is a list.  Each element is a name that corresponds 
    to a level in the final nested dictionary.  
    """

    #Get first name from path
    this_name = path.pop(0)
    
    #Does the element exist already?
    element = find_element(nest["children"], this_name)
    
    #If the element exists, we can use it, otherwise we need to create a new one
    if element:
        
        if len(path)>0:
            add_node(path,value, element)
        
    #Else it does not exist so create it and return its children
    else:
        
        if len(path) == 0:
            nest["children"].append({"name": this_name, "value": value})
        else:
            #Add new element
            nest["children"].append({"name": this_name, "children":[]})
            
            #Get added element 
            element = nest["children"][-1]
            
            #Still elements of path left so recurse
            add_node(path,value, element)
    

In [8]:
d = {"name": "root",
"children": []}

In [9]:
levels = ["l1","l2", "l3"]
for row in df.iterrows():
    r = row[1]
    path = list(r[levels])
    value = r["val"]
    add_node(path,value,d)

In [10]:
#print json.dumps(d, sort_keys=False, indent=2)

In [11]:
#Now create this for the real data
import sqlite3
conn = sqlite3.connect(r"C:\Users\robin_000\Desktop\HMRCTradeDataParser\website\trade_data.db")


In [12]:
sql = """
select code8,code6,code4,code2, mk_commodity_alpha_all as desc8, l6.desc as desc6, l4.desc as desc4, l2.desc as desc2, quantity from 

(select product_code as code8, 
substr(product_code,1,6) as code6,
substr(product_code,1,4) as code4,
substr(product_code,1,2) as code2,
sum(quantity) as quantity
from der_imports_country_products_port_month_8
where year = 2014
group by product_code) as s

left join eightdigitcodes as l8
on s.code8 = l8.mk_comcode8

left join lookup_codes_6 as l6
on s.code6 = l6.code

left join lookup_codes_4 as l4
on s.code4 = l4.code

left join lookup_codes_2 as l2
on s.code2 = l2.code
"""

df_all = pd.read_sql(sql, conn)

In [16]:
d_all = {"name": "root",
"children": []}

levels = ["code2","code4", "code6", "code8"]
for row in df_all.iterrows():
    r = row[1]
    path = list(r[levels])
    value = r["quantity"]
    add_node(path,value,d_all)

In [20]:
import json
with open('data.txt', 'w') as outfile:
    #json.dump(d_all, outfile, sort_keys=False, indent=2)
    json.dump(d_all, outfile)
